## 02. Redux로 전역 상태 관리
- **React는 기본적으로 컴포넌트 내부에서만 상태를 관리하며, 여러 컴포넌트에서 상태를 함께 사용하는 경우에는 전역 상태 관리에 한계 $\rightarrow$ Redux**
- Redux는 자바스크립트 애플리케이션에서 상태 관리를 도와주는 라이브러리
  
  하지만, Redux 는 설정이 복잡하고 문법이 어려우며 작성해야 할 코드량이 많음 $\rightarrow$ Redux Toolkit 사용

<br>

| **메서드** | **설명** |
| - | - |
| `createSlice()` | 리듀서와 액션을 한 번에 생성 |
| `configureStore()` | Redux 스토어를 간편하게 설정 |
| `createAsyncThunk()` | 비동기 작업을 처리하는 액션 생성기 |
| `createEntityAdapter()` | 리스트 데이터 관리를 위한 유틸리티 |

<br>

### Redux, Redux-Toolkit 설치
- `@reduxjs/toolkit` : Redux를 더 쉽고 효율적으로 사용할 수 있는 공식 도구 모음
  - Redux Core (상태 관리 핵심), redux-thunk (비동기 작업 처리), immer (불변성 관리), redux-devtools-extension (개발자 도구 연동), reselect (선택자 함수 지원) 등의 라이브러리 포함
- `react-redux` 

<br>

```bash
$ npm install @reduxjs/toolkit react-redux
```

<br>

### Redux 스토어 생성
- Redux에서는 애플리케이션 전체 상태를 스토어에서 중앙집중식으로 관리
- **스토어 : 애플리케이션에서 사용하는 모든 상태를 한 곳에 모아 저장하고 관리하는 역할을 하는 객체**
  - 애플리케이션에 1개만 존재 가능
  
<br>

- **`configureStore()`**
    - **`reducer` : 상태를 어떻게 변경할지 정의하는 리듀서 함수를 지정**

```ts
const store = configureStore({
    reducer: rootReducer
});
```

<br>

- `src/store/store.ts`
    - **`configureStore()` 함수에 객체를 전달하고, 그 안에 `reducer` 속성을 정의**
    - **`store.getState()` : 스토어의 현재 상태를 반환하는 디스패치 함수**
      - **디스패치 : 스토어에 어떤 액션이 발생했다고 알려주는 함수, 이 함수를 사용해 리듀서가 상태를 변경**
    - **`store.dispatch()` : 액션을 보낼 때 사용하는 함수**

```ts
import configureStore from "../utils/configureStore";

const store = configureStore({
    reducer: {} // 리듀서 등록 위치
});

export type RootState = ReturnType<typeof store.getState>;
export type AppDispatch = typeof store.dispatch; 
```

<br>

### Redux 스토어에 React 제공
- **Redux 스토어를 React 애플리케이션에 사용하려면 `Provider` 컴포넌트로 제공**
- `main.tsx`
    - **`Provider`로 감싸고, `store`를 속성으로 전달 $\rightarrow$ `App` 컴포넌트와 그 하위에 있는 모든 컴포넌트에서 스토어에 접근 가능**

```ts
import { StrictMode } from 'react';
import { createRoot } from 'react-dom/client';
import { store } from './store/store.ts';
import { Provider } from 'react-redux';

import './index.css'
import App from './App.tsx'

createRoot(document.getElementById('root')!).render(
  <StrictMode>
      <Provider store={store}>    
        <App />
      </Provider>
  </StrictMode>,
)
```

<br>

### Redux 상태 슬라이스 
- **슬라이스 (Slice) : 하나의 상태와 해당 상태를 변경하는 액션(리듀서)를 묶은 단위**
- **`createSlice()`** 
  - **액션 타입** : 슬라이스 이름과 리듀서 이름을 결합한 고유 문자열
  - **액션 생성 함수** : 액션 객체를 자동으로 생성하는 함수
  - **리듀서 함수** : 상태를 실제로 변경하는 함수

```ts
const exampleSlice = createSlice({
    name: '슬라이스_이름', // 액션 타입의 접두어로 사용
    initialState : 초깃값, // 상태 초깃값
    reducers: {
        리듀서_이름(state, action) {
            // 상태 변경 로직
        }
    }
})
```

<br>

- `src/store/slice/counterSlice.ts`

```ts
import createSlice from '@reduxjs/toolkit';

// 슬라이스에서 관리한 상태의 구조 정의
export interface CounterState {
    value: number;
}

// 상태의 초깃값
const initialState: CounterState = {
    value: 0,
};

// 슬라이스 생성
export const counterSlice = createSlice({
    name: 'counter',
    initialState,
    // 리듀서 정의
    reducers: {
        increment: (state) => {
            state.value += 1;
        },
        decrement: (state) => {
            state.value -= 1;
        },
        incrementByAmount: (state, action) => {
            state.value += action.payload;
        },
    },
});

export const { increment, decrement, incrementByAmount } = counterSlice.actions;
export default counterSlice.reducer;
```

<br>

### 슬라이스를 스토어에 추가
- **Redux는 모든 상태를 스토어에서 관리하므로, 슬라이스를 스토어에 등록해야 실제로 상태를 조회하거나 변경 가능**
- `src/store/store.ts`

```ts
import configureStore from "../utils/configureStore";
import counterSlice from "./slice/counterSlice";

const store = configureStore({
    reducer: {
        counter: counterSlice
    } // 리듀서 등록 위치
});

export type RootState = ReturnType<typeof store.getState>;
export type AppDispatch = typeof store.dispatch;
```

<br>

### 스토어 사용
- **더 이상 `props`로 상태를 전달할 필요 없이 각 컴포넌트가 Redux 스토어에서 직접 상태를 조회하고 변경 가능**

<br>

1. **`App.tsx`, `Count.tsx`의 `props` 전달 코드 제거**
- `src/components/Count.tsx`

```ts
import Count from "./components/Count";
import CountOutsideDisplay from "./components/CountOutsideDisplay";

export default function App() {
  return (
    <>
      <Count />
      <CountOutsideDisplay />
    </>
  );
}
```

<br>

- `src/components/Count.tsx`

```ts
import CountButtons from "./CountButtons";
import CountDisplay from "./CountDisplay";

export default function Count() {
  return (
    <>
      <CountDisplay />
      <CountButtons />
    </>
  );
}
```

<br>

2. **`CountOutsideDisplay` 컴포넌트에서 스토어 상태를 조회하도록 수정**
- **`useSelector` : 스토어에 저장된 상태를 컴포넌트에서 쉽게 조회할 수 있게 하는 조회 전용 훅** 
- **`RootState` : 스토어의 전체 상태 타입으로, `store.ts`파일에 정의됨**
- **`state.counter.value` : 스토어에 `counter` 라는 이름으로 등록한 슬라이스의 `value` 값 상태**

```ts
import { useSelector } from "react-redux";
import { type RootState } from "../store/store";

export default function CountOutsideDisplay() {
  const count = useSelector((state: RootState) => state.counter.value);
  return <h1>Outside Count: {count}</h1>;
}
```

<br>

- `src/components/CountDisplay.tsx`

```ts
import { useSelector } from "react-redux";
import { type RootState } from "../store/store";

export default function CountDisplay() {
  const count = useSelector((state: RootState) => state.counter.value);
  return <h1>Count: {count}</h1>;
}
```

<br>

- `src/components/CountButtons.tsx`
    - **`useDispatch()` : 스토어의 디스패치 함수를 가져옴**
    - **`dispatch(액션_생성_함수())`** : Redux가 해당 액션을 처리할 리듀서를 자동으로 실행

```ts
import { useDispatch } from "react-redux";

// 슬라이스의 액션 생성 함수
import {
  decrement,
  increment,
  reset,
} from "../store/slice/counterSlice";

export default function CountButtons() {
  const dispatch = useDispatch();
  return (
    <>
      <button onClick={() => dispatch(decrement())}>감소</button>
      <button onClick={() => dispatch(reset())}>초기화</button>
      <button onClick={() => dispatch(increment())}>증가</button>
        10 증가
      </button>
    </>
  );
}
```

<br>

### 값을 전달해 상태 변경하기

- **리듀서 함수의 두 번째 매개변수 `action`**
- `src/store/slice/counterSlice.ts`
  - **입력받은 값만큼 상태에 반영하는 리듀서 함수 정의**

```ts
import { createSlice, type PayloadAction } from "@reduxjs/toolkit";

export interface CounterState {
  value: number;
}
const initialState: CounterState = {
  value: 0,
};
export const counterSlice = createSlice({
  name: "counter",
  initialState,
  reducers: {
    increment: (state) => {
      state.value += 1;
    },
    decrement: (state) => {
      state.value -= 1;
    },
    reset: (state) => {
      state.value = 0;
    },

    // 입력받은 값만큼 상태에 반영.
    incrementByAmount: (state, action: PayloadAction<{ count: number }>) => {
      state.value += action.payload.count;
    },
  },
});

export const { increment, decrement, reset, incrementByAmount } =
  counterSlice.actions;
export default counterSlice.reducer;
```

<br>

- `src/components/CountButtons.tsx`

```ts
import { useDispatch } from "react-redux";
import {
  decrement,
  increment,
  incrementByAmount,
  reset,
} from "../store/slice/counterSlice";

export default function CountButtons() {
  const dispatch = useDispatch();
  return (
    <>
      <button onClick={() => dispatch(decrement())}>감소</button>
      <button onClick={() => dispatch(reset())}>초기화</button>
      <button onClick={() => dispatch(increment())}>증가</button>
      <button onClick={() => dispatch(incrementByAmount({ count: 10 }))}>
        10 증가
      </button>
    </>
  );
}
```

<br>

<img src='img/1203.png' width=400>

<br>

<hr>